# Airfoil Analysis Notebook

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
import numpy as np

from ICARUS.core.units import calc_mach, calc_reynolds
from ICARUS.airfoils import Airfoil
from ICARUS.core.base_types import Struct
from ICARUS.computation.solvers import Solver
from ICARUS.computation.solvers.OpenFoam.files.setup_case import MeshType

# Load DB

In [ ]:
from ICARUS.database import Database

# CHANGE THIS TO YOUR DATABASE FOLDER
database_folder = "E:\\Icarus\\Data"

# Load the database
DB = Database(database_folder)
DB.load_all_data()

In [ ]:
print(f"Total number of loaded airfoils {len(list(DB.airfoils.keys()))}")
print(
    f"Total number of computed airfoil data {len(list(DB.foils_db._raw_data.keys()))}"
)
print(f"Total number of computed airfoil polars {len(list(DB.airfoil_polars.keys()))}")

# Reynolds And Mach and AoA

In [ ]:
# Assumptions
chord_max: float = 0.5
chord_min: float = 0.1
u_max: float = 35.0
u_min: float = 5.0
viscosity: float = 1.56e-5
speed_of_sound: float = 340.3

# MACH ESTIMATION
mach_min: float = calc_mach(10, speed_of_sound)
mach_max: float = calc_mach(30, speed_of_sound)
mach = np.linspace(mach_max, mach_min, 10)
MACH: float = mach_max
print(f"Mach range: {mach_min} - {mach_max}")
print(f"Mach:\n{mach}\n")

# REYNOLDS ESTIMATION
reynolds_max: float = calc_reynolds(u_max, chord_max, viscosity)
reynolds_min: float = calc_reynolds(u_min, chord_min, viscosity)
reynolds = np.linspace(reynolds_min, reynolds_max, 20)
# reynolds = [1e5, 10e6]

REYN = reynolds_max
print(f"Reynolds range: {reynolds_min} - {reynolds_max}")
print(f"Re:\n{reynolds}\n")
# ANGLE OF ATTACK SETUP
aoa_max: float = 12
aoa_min: float = -10
num_of_angles: float = (aoa_max - aoa_min) * 2 + 1
angles = np.linspace(
    start=aoa_min,
    stop=aoa_max,
    num=num_of_angles,
)

print(f"Angles of attack:\n{angles}\n")

ftrip_low: dict[str, float] = {"pos": 0.1, "neg": 0.2}
ftrip_up: dict[str, float] = {"pos": 0.2, "neg": 0.1}

In [ ]:
cleaning = False
calcF2W = True
calcOpenFoam = False
calcXFoil = True

# Get Airfoils

In [ ]:
all_airfoils = list(DB.airfoils.keys())

airfoils_to_compute = [
    airfoil
    for airfoil in all_airfoils
    if (
        # airfoil.upper().startswith("AG")  # or
        # airfoil.upper().startswith('CLARK') or
        # airfoil.upper().startswith('DAE') or
        # airfoil.upper().startswith('E') or
        # airfoil.upper().startswith('H') or
        # airfoil.upper().startswith('M') or
        # airfoil.upper().startswith('N') or
        # airfoil.upper().startswith('O') or
        # airfoil.upper().startswith('W')
    )
]
airfoils_to_compute = ["NACA4415", "NACA4412"]
print(len(airfoils_to_compute))

# Xfoil

In [ ]:
if calcXFoil:
    from ICARUS.computation.solvers.Xfoil.xfoil import Xfoil

    for airfoil in airfoils_to_compute:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get airfoil
        airf: Airfoil = DB.get_airfoil(airfoil)
        # airf.plot(scatter=True)
        # airf.repanel(120, distribution="cosine")
        xfoil: Solver = Xfoil()

        # Import Analysis
        analysis: str = xfoil.get_analyses_names()[1]  # Run
        xfoil.select_analysis(analysis)

        # Get Options
        options = xfoil.get_analysis_options(verbose=False)
        solver_parameters = xfoil.get_solver_parameters()

        # Set Options
        options.airfoil = airf
        options.mach = MACH
        options.reynolds = reynolds
        options.min_aoa = aoa_min
        options.max_aoa = aoa_max
        options.aoa_step = 0.5

        # Set Solver Options
        solver_parameters.max_iter = 1000
        solver_parameters.Ncrit = 9
        solver_parameters.xtr = (0.1, 0.2)
        solver_parameters.print = False

        xfoil.define_analysis(options, solver_parameters)
        xfoil.print_analysis_options()

        # RUN
        xfoil.execute(parallel=False)
        # Get polar
        polar = DB.get_airfoil_polars(airf, "Xfoil")
        polar.plot()

In [ ]:
polar = DB.get_airfoil_polars(airfoils_to_compute[1], "Xfoil")
ax = polar.plot()

# Foil2Wake

In [ ]:
if calcF2W:
    # Import Solver
    from ICARUS.computation.solvers.Foil2Wake.f2w_section import Foil2Wake

    f2w_s: Solver = Foil2Wake()

    for airfoil in airfoils_to_compute:
        print(f"\nRunning airfoil {airfoil}\n")
        # Import Analysis
        analysis: str = f2w_s.get_analyses_names()[1]  # Run
        f2w_s.select_analysis(analysis)

        # Get Options
        options: Struct = f2w_s.get_analysis_options(verbose=False)
        solver_parameters: Struct = f2w_s.get_solver_parameters()

        # Set Options
        options.airfoil = DB.get_airfoil(airfoil)
        options.reynolds = reynolds
        options.mach = MACH
        options.angles = angles

        solver_parameters.f_trip_upper = 0.03
        solver_parameters.f_trip_low = 0.1

        # RUN
        f2w_s.define_analysis(options, solver_parameters)
        f2w_s.print_analysis_options()
        f2w_s.execute(parallel=True)

        # Get Results
        polars = f2w_s.get_results()

# Mapflow

In [ ]:
# Not Implemented

# OpenFoam (NOT HIGH MACH)

In [ ]:
if calcOpenFoam:
    airfoils_to_compute: list[str] = [airf.name]  # ["0008"]
    from ICARUS.computation.solvers.OpenFoam.open_foam import OpenFoam

    for airfoil in airfoils_to_compute:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get Airfoil
        airf: Airfoil = Airfoil.naca(airfoil, n_points=200)
        # airf.plot()
        open_foam: Solver = OpenFoam()

        # Import Analysis
        analysis: str = open_foam.get_analyses_names()[0]  # Run
        open_foam.select_analysis(analysis)

        # Get Options
        options = open_foam.get_analysis_options(verbose=True)
        solver_parameters = open_foam.get_solver_parameters()

        # Set Options
        options.airfoil = airf
        options.angles = angles
        options.reynolds = REYN
        options.mach = MACH
        open_foam.print_analysis_options()

        # Set Solver Options
        solver_parameters.mesh_type = MeshType.structAirfoilMesher
        solver_parameters.max_iterations = 20
        solver_parameters.silent = False

        # RUN
        open_foam.set_analysis_options(options)
        open_foam.print_analysis_options()
        open_foam.execute()